# Tutorial 04: Particle Methods

Learn how to use particle-based solvers as an alternative to grid-based methods for solving the Fokker-Planck equation.

## Learning Objectives

By the end of this tutorial, you will understand:
- The difference between grid-based (FDM) and particle-based solvers
- How to configure particle methods using `ConfigBuilder`
- When to use particles vs grids
- How Kernel Density Estimation (KDE) works
- Trade-offs between accuracy and computational cost

## Mathematical Background

### Grid-Based Methods (FDM)

Standard approach: Discretize the Fokker-Planck PDE on a grid

$$\frac{\partial m}{\partial t} - \sigma^2 \frac{\partial^2 m}{\partial x^2} + \frac{\partial}{\partial x}\left(m \frac{\partial H}{\partial p}\right) = 0$$

**Advantages**: Deterministic, well-understood convergence

**Disadvantages**: Exponential cost in high dimensions (curse of dimensionality)

### Particle Methods

Alternative approach: Simulate individual agents as stochastic particles

$$dX_t = -\nabla_p H(X_t, \nabla u, m) dt + \sigma dW_t$$

**Advantages**: Scales linearly with dimension, natural for complex geometries

**Disadvantages**: Statistical noise, requires many particles for accuracy

### Kernel Density Estimation (KDE)

Convert particles to continuous density:

$$m(x) \approx \frac{1}{N} \sum_{i=1}^N K_h(x - X_i)$$

where $K_h$ is a kernel with bandwidth $h$.

**Time estimate**: 25 minutes

## Step 1: Import Dependencies

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from mfg_pde import MFGProblem, solve_mfg
from mfg_pde.factory import ConfigBuilder
from mfg_pde.geometry import TensorProductGrid

## Step 2: Create Test Problem

We'll use the same simple Linear-Quadratic problem for both methods to compare their performance.

In [ ]:
print("=" * 70)
print("TUTORIAL 04: Particle Methods")
print("=" * 70)
print()

# Create domain using TensorProductGrid
domain = TensorProductGrid(
    dimension=1,
    bounds=[(0.0, 1.0)],
    num_points=[51],  # 51 points = 50 intervals
)

# Create MFG problem
problem = MFGProblem(
    geometry=domain,
    T=1.0,
    Nt=50,
    sigma=0.15,  # Higher diffusion shows particle noise more clearly
    lam=0.3,  # Congestion weight
)

print("Problem configuration:")
print("  Domain: [0, 1]")
print(f"  Grid: {problem.Nx} points")
print(f"  Time steps: {problem.Nt}")
print(f"  Diffusion sigma = {problem.sigma}")
print()

## Step 3: Solve with Grid-Based Methods

First, we'll solve using the default grid-based (FDM) approach for both HJB and FP equations.

In [ ]:
print("METHOD 1: Grid-Based (FDM for both HJB and FP)")
print("-" * 70)
print()

# Configure grid-based solver
config_grid = (
    ConfigBuilder()
    .picard(max_iterations=30, tolerance=1e-4)
    .solver_hjb("fdm")  # Finite Difference Method for HJB
    .solver_fp("fdm")  # Finite Difference Method for FP
    .build()
)

result_grid = solve_mfg(problem, config=config_grid, verbose=True)

print()
print("Grid-based solution:")
print(f"  Converged: {result_grid.converged}")
print(f"  Iterations: {result_grid.iterations}")
print(f"  Final error: {result_grid.max_error:.6e}")
print()

## Step 4: Solve with Particle Methods

Now we'll use a **hybrid approach**:
- **HJB**: Still use FDM (particles don't solve backward PDEs well)
- **FP**: Use particle simulation with KDE

This is the recommended combination for production use.

In [ ]:
print("METHOD 2: Hybrid (FDM for HJB, Particles for FP)")
print("-" * 70)
print()

# Configure particle solver
config_particle = (
    ConfigBuilder()
    .picard(max_iterations=30, tolerance=1e-4)
    .solver_hjb("fdm")  # HJB: Finite Difference Method
    .solver_fp(
        "particle",
        num_particles=5000,  # Number of simulated agents
        kde_bandwidth="scott",  # Automatic bandwidth selection
    )
    .build()
)

print("Configuration:")
print("  HJB solver: FDM (grid-based)")
print("  FP solver: Particle-based")
print("  Number of particles: 5,000")
print("  KDE bandwidth: Scott's rule (automatic)")
print()

result_particle = solve_mfg(problem, config=config_particle, verbose=True)

print()
print("Particle-based solution:")
print(f"  Converged: {result_particle.converged}")
print(f"  Iterations: {result_particle.iterations}")
print(f"  Final error: {result_particle.max_error:.6e}")
print()

## Step 5: Compare Solutions

Let's quantify the difference between the two methods.

In [ ]:
print("=" * 70)
print("COMPARISON: Particle vs Grid")
print("=" * 70)
print()

# L2 error between solutions
density_error = np.linalg.norm(result_particle.M - result_grid.M) / np.linalg.norm(result_grid.M)
value_error = np.linalg.norm(result_particle.U - result_grid.U) / np.linalg.norm(result_grid.U)

print("Relative errors (particle vs grid):")
print(f"  Density (M): {density_error:.4%}")
print(f"  Value (U):   {value_error:.4%}")
print()

# Convergence comparison
print("Convergence comparison:")
print(f"  Grid iterations:     {result_grid.iterations}")
print(f"  Particle iterations: {result_particle.iterations}")
print()

if density_error < 0.05:
    print("✓ Particle method agrees well with grid method (<5% error)")
else:
    print(f"⚠ Higher error ({density_error:.1%}) - may need more particles")
print()

## Step 6: Visualize Comparison

Plot both solutions side-by-side.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

x = problem.xSpace

# Final density comparison
axes[0].plot(x, result_grid.M[-1, :], "b-", linewidth=2, label="Grid (FDM)")
axes[0].plot(x, result_particle.M[-1, :], "r--", linewidth=2, label="Particle", alpha=0.7)
axes[0].set_xlabel("x")
axes[0].set_ylabel("Density m(T, x)")
axes[0].set_title("Final Density Comparison")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Error at each grid point
pointwise_error = np.abs(result_particle.M[-1, :] - result_grid.M[-1, :])
axes[1].plot(x, pointwise_error, "g-", linewidth=2)
axes[1].set_xlabel("x")
axes[1].set_ylabel("Absolute Error")
axes[1].set_title("Pointwise Error Distribution")
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Mean pointwise error: {np.mean(pointwise_error):.6e}")
print(f"Max pointwise error:  {np.max(pointwise_error):.6e}")

## Summary

### What You Learned

1. **Two approaches to FP equation**:
   - Grid-based (FDM): Deterministic, grid-based discretization
   - Particle-based: Stochastic simulation with KDE

2. **Hybrid configuration** (recommended):
   - HJB: FDM (backward PDE, needs grid)
   - FP: Particles (forward evolution, natural for high-D)

3. **Trade-offs**:
   - Accuracy: More particles → lower error (~O(1/√N))
   - Speed: More particles → longer runtime
   - Dimension: Particles scale better to high-D

4. **When to use particles**:
   - High-dimensional problems (D ≥ 3)
   - Complex geometries with obstacles
   - When statistical noise is acceptable
   - Large-scale agent simulations

5. **When to use grids**:
   - Low dimensions (1D, 2D)
   - Need deterministic convergence
   - Smooth, simple geometries
   - High accuracy requirements

### Key Takeaways

- **Hybrid approach**: Best of both worlds (FDM for HJB, particles for FP)
- **Particle count matters**: 5,000-10,000 particles typically sufficient for 1D/2D
- **KDE bandwidth**: Use automatic selection ("scott" or "silverman")
- **Curse of dimensionality**: Particles escape it, grids don't

### Next Steps

Proceed to **Tutorial 05: ConfigBuilder System** to master advanced solver configuration.